In [1]:
import pandas as pd

APP_NAME = 'pyspark_python'
MASTER = 'local[*]'
from pyspark import SparkConf
from pyspark.sql import SparkSession


conf = SparkConf().setAppName(APP_NAME)
conf = conf.setMaster(MASTER)
spark = SparkSession.builder.config(conf = conf).getOrCreate()
sc = spark.sparkContext

## JOINs

**Join is one of the most expensive operations you will commonly use in Spark, so it is worth doing what you can to shrink your data before performing a join**

As with most operations, the cost of the join increases with the number of keys and the distance the records have to travel in order to get to their correct partition.

## **Choosing a Join Type**

The default join operation in Spark includes only values for keys present in both dataframes, and in the case of multiple values per key, provides all permutations of the key pair. The best scenario for a standard join is when both dataframes contain the same set of distinct keys. With duplicate keys, the size of the data may expand dramatically causing performance issues, and if one key is not present in both data.frames you will lose that row of data. 

Here are a few guidelines:

- When both data.frames have duplicate keys, the join can cause the size of the data to expand dramatically. It may be better to perform a distinct to reduce the key spac instead of producing the full cross product. By using smart partitioning during the combine step, it is possible to prevent a **second shuffle** in the join.

- If keys are not present in both dataframes you risk losing your data unexpectedly. It can be safer to use an outer join, so that you are guaranteed to keep all the data in either the left or the right join, then filter the data after the join.

- If one data.frame has some easy-to-define subset of the keys, in the other you may be better off filtering or reducing before the join to avoid a big shuffle of data, which you will ultimately throw away anyway.


## Choosing an Execution Plan

- In order to join data, Spark needs the data that is to be joined (i.e., the data based on each key) to live on the same partition. The default implementation of a join in Spark is a shuffled hash join. The shuffled hash join ensures that data on each partition will contain the same keys by partitioning the second dataset with the same default partitioner as the first, so that the keys with the same hash value from both datasets are in the same partition. While this approach always works, it can be more expensive than necessary because it requires a shuffle. The shuffle can be avoided if:

- Both dataframes have a known partitioner.

- One of the datasets is small enough to fit in memory, in which case we can do a broadcast hash join.

Note that if the dataframes are colocated the network transfer can be avoided, along with the shuffle.

https://www.oreilly.com/library/view/high-performance-spark/9781491943199/ch04.html